# ResponseDataWidget Demonstration

This notebook demonstrates the usage of the `ResponseDataWidget` class for interactive visualization of response data from material simulations.

The widget allows selecting various variables for plotting on the X and Y axes, including:

- Time (t)
- Primary variables (strain, stress)
- Internal variables (damage, plastic strain, etc.)
- Conjugate thermodynamic forces

We'll use a GSM1D_ED (1D Elastic Damage) model to create some response data and then visualize it with our widget.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import traits.api as tr
import bmcs_utils.api as bu
import ipywidgets as widgets

# Import the GSM framework
from bmcs_matmod.gsm_lagrange.core.gsm_model import GSMModel

# Import the ResponseDataWidget
from bmcs_matmod.gsm_lagrange.core.response_data_widget import ResponseDataWidget

# Import the specific model we'll use for demonstration
from bmcs_matmod.gsm_lagrange.models.gsm1d_ed import GSM1D_ED

# For nicer plot display
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 2
try:
    plt.style.use('bmcs')
except:
    pass

# For LaTeX rendering in plots
plt.rcParams['text.usetex'] = False

ModuleNotFoundError: No module named 'bmcs_matmod.gsm_lagrange.core.response_data_widget'

## 1. Create a Material Model

First, let's create a material model with specific parameters. We'll use the elastic-damage model (GSM1D_ED) for demonstration.

In [ ]:
# Create a material model from the GSM1D_ED symbolic model
ed_material = GSMModel(GSM1D_ED)

# Set parameters to customize the model using set_params
ed_material.set_params(
    E=20000.0,    # Young's modulus (MPa)
    S=1,          # Damage threshold strain
    c=1,          # ratio
    eps_0=0.0     # Initial inelastic strain
)

# Display model parameters
print("Model parameters:")
for param_sym, name in ed_material.trait_model_params.items():
    value = getattr(ed_material, name)
    print(f"  {name} = {value} ({param_sym})")

## 2. Generate Response Data

Now, let's generate response data by simulating a monotonic tensile test. This will create a `ResponseData` object containing the time history of all variables.

In [ ]:
# Define a monotonic tensile strain history
n_steps = 1000
strain_max = 0.13  # Maximum strain
strain = np.linspace(0, strain_max, n_steps)
time = np.linspace(0, 1.0, n_steps)

# Run the simulation with our material model
rd = ed_material.get_F_response(strain, time)

# Display information about the response data
print(f"Time steps: {len(rd.t_t)}")
print(f"Primary variables:\n  strain shape: {rd.eps_t.shape}\n  stress shape: {rd.sig_t.shape}")
print(f"Internal variables: {list(rd.Eps_t.keys())}")
print(f"Conjugate forces: {list(rd.Sig_t.keys())}")

## 3. Traditional Static Plotting

Let's first look at how we would traditionally plot some of this data using matplotlib directly.

In [ ]:
# Extract and reshape for easier plotting
eps = rd.eps_t[:, 0]
sig = rd.sig_t[:, 0]
omega = rd.Eps_t['omega_a'][:, 0, 0]  # Internal damage variable

# Plot the stress-strain curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Stress-strain curve
ax1.plot(eps, sig, 'b-', linewidth=2)
ax1.set_xlabel('Strain $\\varepsilon$')
ax1.set_ylabel('Stress $\\sigma$ (MPa)')
ax1.set_title('Stress-Strain Curve (Tensile Test)')
ax1.grid(True)

# Evolution of internal variable (omega - damage)
ax2.plot(eps, omega, 'g-', linewidth=2)
ax2.set_xlabel('Strain $\\varepsilon$')
ax2.set_ylabel('Damage variable $\\omega$')
ax2.set_title('Evolution of Damage Variable')
ax2.grid(True)

plt.tight_layout()

## 4. Interactive Plotting with ResponseDataWidget

Now, let's use our `ResponseDataWidget` to plot the same data interactively. This widget allows us to select any variable for the x and y axes.

In [ ]:
# Create the widget with our response data
widget = ResponseDataWidget(response_data=rd)

# Display the widget
widget.get_widget()

## 5. Exploring Different Relationships

With the interactive widget above, you can now explore different relationships between variables by selecting different options for the X and Y axes. Try exploring relationships like:

- Stress vs. Strain (standard stress-strain curve)
- Damage vs. Strain (evolution of damage during loading)
- Stress vs. Damage (relationship between stress and damage)
- Time vs. any variable (evolution over time)

This interactive exploration helps in understanding the material behavior better without having to create multiple static plots.

```python
# Set matplotlib backend
%matplotlib inline

# Create the widget with simple initialization steps
widget = ResponseDataWidget()

# Set the response data
widget.response_data = rd

# Get the widget
w = widget.get_widget()

# Display the widget
display(w)

# Now that widget is displayed, directly update dropdown values
# to make sure they're set correctly and trigger updates
widget.x_dropdown.value = 'eps_t_0'
widget.y_dropdown.value = 'sig_t_0'
```

In [ ]:
# DEBUGGING CELL: Direct visualization to check if the data and plotting work
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
eps = rd.eps_t[:, 0]
sig = rd.sig_t[:, 0]

print(f"eps shape: {eps.shape}, sig shape: {sig.shape}")
print(f"eps min/max: {eps.min()}/{eps.max()}")
print(f"sig min/max: {sig.min()}/{sig.max()}")

plt.plot(eps, sig, 'r-', linewidth=2)
plt.xlabel('Strain ε[0]')
plt.ylabel('Stress σ[0]')
plt.title('Direct Matplotlib Plot (Debug)')
plt.grid(True)
plt.show()

# Try another direct approach with our widget's figure
plt.figure(widget.fig.number)
plt.clf()
plt.plot(eps, sig, 'g-', linewidth=2)
plt.xlabel('Strain')
plt.ylabel('Stress')
plt.title('Another test using widget figure')
plt.grid(True)
plt.show()

In [ ]:
# Activate matplotlib for the notebook
%matplotlib inline

# Create a simple direct plot as a sanity check
plt.figure(figsize=(10, 6))
eps = rd.eps_t[:, 0]
sig = rd.sig_t[:, 0]
plt.plot(eps, sig, 'b-', linewidth=2)
plt.xlabel('Strain ε[0]')
plt.ylabel('Stress σ[0]')
plt.title('Direct Plot (no widget)')
plt.grid(True)
plt.show()

## 6. Creating a Response Data Widget From Scratch

If you want to create your own response data widget with more customization, you can do so as follows:

In [ ]:
# Create a custom response data widget
custom_widget = ResponseDataWidget()

# Get the widget interface (this initializes the figure/axes)
custom_w = custom_widget.get_widget()

# Set specific default selections for this widget
custom_widget.x_var = 'eps_t_0'  # Strain as x-axis
custom_widget.y_var = 'Eps_t_omega_a_0'  # Damage as y-axis (if available)

# Now assign the response data
custom_widget.response_data = rd

# Force a plot update
custom_widget.update_plot()

# Display the widget
custom_w